In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle

In [2]:
# Load Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/Minor Project - Semester 7/global_model.sav"

In [5]:
# Taking pickled model

In [6]:
model = pickle.load(open(path, 'rb'))

In [7]:
model.coef_

array([ 0.06825225, -0.91072404,  0.38586088, -0.0498874 ,  0.56109946,
        0.99988492,  0.87289432])

In [8]:
model.intercept_

array(-7.6449437e-16)

In [9]:
# User Input
# Example: 26, 66.58, 1.63, F, 25.02, 1449.35, 1.9

In [33]:
age = 26
height = 1.63
weight = 66
gender = 'F'
activity_level = 1.9

In [34]:
# Encode Gender
# 0 - Female
# 1 - Male

if gender == 'F':
  gender = 0
else:
  gender = 1

In [36]:
# BMI calculation

bmi = round(weight/((height)**2),2)
bmi

24.84

In [40]:
# BMR calculation

if gender == 1:
    bmr = 10*weight+6.25*height*100-5*age+5
else:
    bmr = 10*weight+6.25*height*100-5*age-161

bmr

1387.75

In [41]:
# Model input

In [57]:
user_input = np.array([age, weight, height, gender, bmi, bmr, activity_level]).reshape(1, -1)

In [58]:
user_input

array([[  26.  ,   66.  ,    1.63,    0.  ,   24.84, 1387.75,    1.9 ]])

In [59]:
calorie_prediction = model.predict(user_input)

In [60]:
calorie_prediction

array([1345.48223509])

In [ ]:
# Calories per meal

In [61]:
calorie_dist_meal_wise = {'breakfast':0.35, 'lunch':0.40, 'dinner': 0.45}

In [ ]:
# Load recipes dataset

In [62]:
path = "/content/drive/MyDrive/Minor Project - Semester 7/Datasets/recipes_processed.csv"

In [67]:
df = pd.read_csv(path)

In [68]:
df.columns

Index(['Name', 'Calories', 'FatContent', 'SaturatedFatContent',
       'CholesterolContent', 'SodiumContent', 'CarbohydrateContent',
       'FiberContent', 'SugarContent', 'ProteinContent'],
      dtype='object')

In [69]:
from sklearn.neighbors import NearestNeighbors

nn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
nn_model.fit(df.iloc[:,1:])

NearestNeighbors(algorithm='brute', metric='cosine')

In [70]:
predictor = nn_model.kneighbors

In [71]:
# Make list of nutrition values

import random

max_calorie = 2000
max_fat = 100
max_saturated_fat = 13
max_cholesterol = 300
max_sodium = 2300
max_carbohydrate = 325
max_fiber = 40
max_sugar = 40
max_protein = 200
max_list = [max_calorie,max_fat,max_saturated_fat,max_cholesterol,max_sodium,max_carbohydrate,max_fiber,max_sugar,max_protein]

nutrition_values = np.array([random.randint(1, max_val) for max_val in max_list])

In [76]:
for key, value in calorie_dist_meal_wise.items():
  print(key + " ----------------------------->>")

  calorie_intake_in_meal = calorie_prediction * value

  # Replace calorie in nutrition_values with calorie_intake_in_meal
  nutrition_values[0] = calorie_intake_in_meal

  nearest_neighbors = predictor(nutrition_values.reshape(1, -1), n_neighbors = 5, return_distance = False)
  output_df = df.iloc[nearest_neighbors[0]]
  print(output_df.iloc[:,0])
  print()

breakfast ----------------------------->>
453097     Brook&rsquo;s Combo Fried Rice
216861    Pork Chops With Plum Rose Sauce
505400                  Ming's Pork Ramen
433381          Pretzel Bites With Cheese
95315                    Red Pepper Brine
Name: Name, dtype: object

lunch ----------------------------->>
453097     Brook&rsquo;s Combo Fried Rice
216861    Pork Chops With Plum Rose Sauce
505400                  Ming's Pork Ramen
95315                    Red Pepper Brine
437047        Homemade Soft Pretzel Bites
Name: Name, dtype: object

dinner ----------------------------->>
216861            Pork Chops With Plum Rose Sauce
453097             Brook&rsquo;s Combo Fried Rice
505400                          Ming's Pork Ramen
95315                            Red Pepper Brine
435394    Medieval Game Hen Pot Pie - Alton Brown
Name: Name, dtype: object

